In [1]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
from model import DNN
from forward_config import *


In [2]:
model_p_drop = []

device = torch.device(device)
lr = 0.001
epochs = 24000

layers = [200, 200, 200, 200, 1]

L = 5.
M = 5
tol = 0
n_t = 100

u = torch.linspace(Rk/2/r0, Rk/r0, 70)
u = torch.log(u)

u1 = torch.linspace(np.log(rc/r0), np.log(Rk/2/r0), 50)
u = torch.hstack((u, u1[:-1]))
u = torch.sort(u)[0]

t = torch.linspace(t0, 1, n_t)

grid = torch.cartesian_prod(t, u).float().to(device)
grid = grid[grid[:, 0].argsort()]

# p(u,0)=1
bnd1 = torch.cartesian_prod(torch.tensor([0.]), u).float().to(device)
bndval1 = torch.tensor([1.]).to(device)

#bnd2(u(-1), t)=1
bnd2 = torch.cartesian_prod(t, torch.tensor([u[-1].item()])).float().to(device)
bndval2 = torch.tensor([1.]).to(device)

#bnd3
bnd3 = torch.cartesian_prod(t, torch.tensor([u[0].item()])).float().to(device)


#NN = Modified_MLP(layers, L, M, device='cuda')
NN = nn.Sequential(
    nn.Linear(2, 100),
    nn.Tanh(),
    nn.Linear(100, 100),
    nn.Tanh(),
    nn.Linear(100, 100),
    nn.Tanh(),
    nn.Linear(100, 100),
    nn.Tanh(),
    nn.Linear(100, 1)).to(device)


# Initialize neural network
model = DNN(NN, tol, n_t).to(device)

# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ExponentialLR(optimizer, gamma=0.9)

# Train PINNs
def train(epoch, m):
    model.train()

    def closure():
        optimizer.zero_grad()
        L_t, W = model.loss_weights(grid)
        loss_bnd1 = model.loss_dirichlet(bnd1, bndval1).reshape(-1)
        loss_bnd2 = model.loss_dirichlet(bnd2, bndval2).reshape(-1)
        loss_bnd3 = model.loss_operator(bnd3).reshape(-1)
        loss = torch.mean(W*L_t) + 10000*(torch.hstack([loss_bnd1, loss_bnd2, loss_bnd3])).mean()
        loss.backward()
        return loss


    # Optimize loss function
    loss_pde = optimizer.step(closure)
    if epoch % 3000 == 0:
        scheduler.step()
        print(scheduler.get_last_lr())
    loss_value = loss_pde.item() if not isinstance(loss_pde, float) else loss_pde


    if epoch%500==0:
        print(f'epoch {epoch}: loss {loss_value:.6f}')
        fig1 = plt.figure()
        ax1 = fig1.add_subplot(projection='3d')
        ax1.plot_trisurf(grid[:, 1].cpu().detach().numpy().reshape(-1), grid[:, 0].cpu().detach().numpy().reshape(-1),
                    model(grid).cpu().detach().numpy().reshape(-1), cmap=cm.jet, linewidth=0.2, alpha=1)
        ax1.set_xlabel("x1")
        ax1.set_ylabel("x2")
        plt.show()

# Print CPU
print('Start training...')
tic = time.time()
for epoch in range(1, epochs+1):
    if epoch < 200:
        train(epoch, None)
    else:
        train(epoch, 1)

toc = time.time()
print(f'Total training time: {toc - tic}')




Start training...


KeyboardInterrupt: 